# Manipulate atmospheric forcing data to simulate the OTC effect from the field experiment

placeholder for now. Once forcing data is ready, fill in this notebook to make a modified version of the atmospheric forcing that has increased temperature, similar to the experimental OTC effect. 

Compare observational temperature data inside vs outside OTC, apply similar increase to forcing temperature. Upload as new zipped input data folder.